In [99]:
import pandas as pd
import numpy as np
import math
import operator

In [100]:
data = pd.read_csv("bill_authentication.csv")
data.head()

,Variance,Skewness,Curtosis,Entropy,Class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [101]:
# Defining a function which calculates euclidean distance between two data points
def euclideanDistance(data1, data2, length):
    distance = 0
    for x in range(length):
        distance += np.square(data1[x] - data2[x])
    return np.sqrt(distance)

In [102]:
# Defining KNN model
def knn(trainingSet, testInstance, k):
 
    distances = {}
    sort = {}
    
    # Calculate Distances
 
    length = testInstance.shape[1]
    
    for x in range(len(trainingSet)):
        
        dist = euclideanDistance(testInstance, trainingSet.iloc[x], length)
        distances[x] = dist[0]
        
 
    # Sorting them on the basis of distance
    sorted_d = sorted(distances.items(), key=operator.itemgetter(1))
    #### End of STEP 3.2
 
    neighbors = []
    
    # Extracting top k neighbors
    for x in range(k):
        neighbors.append(sorted_d[x][0])

    classVotes = {}
    # Calculating the most freq class in the neighbors
    for x in range(len(neighbors)):
        response = trainingSet.iloc[neighbors[x]][-1]
 
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1

    # Sort Votes
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return(sortedVotes[0][0], neighbors)




In [103]:
msk = np.random.rand(len(data)) < 0.8
train = data[msk]
test = data[~msk]
#test['Class'].sum()

In [104]:
#%%timeit

k = 3
lst= []
for index, row in test.iterrows():
    testSet=  [row["Variance"], row["Skewness"], row["Curtosis"], row["Entropy"]]
    aux = pd.DataFrame(testSet)
    result,neigh = knn(train, aux, k)
    lst.append(result)

    

In [105]:
df1 = pd.DataFrame(lst)
test['Predictions'] = pd.Series(lst, index=test.index)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [106]:
#%%timeit
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train.iloc[:,0:4], train['Class'])
# Predicted class
test['scikit']= neigh.predict(test.iloc[:,0:4])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [112]:
from sklearn import metrics
print('The accuracy of KNN from scratch: ',metrics.accuracy_score(test['Class'], test['Predictions']))
print('The accuracy of KNN from Sci-kit: ',metrics.accuracy_score(test['Class'], test['scikit']))

The accuracy of KNN from scratch:  0.8206896551724138
The accuracy of KNN from Sci-kit:  1.0


In [107]:
metrics.confusion_matrix(test['Class'], test['Predictions'])

array([[132,  34],
       [ 18, 106]])

In [108]:
metrics.confusion_matrix(test['Class'], test['scikit'])

array([[166,   0],
       [  0, 124]])